# Source code

**CS5483 Data Warehousing and Data Mining**
___

Content of `datamining.mac`:

In [1]:
from IPython.display import Code

In [2]:
Code('datamining.mac',language='Maxima')

/*  Author Chung Chan
    Copyright (C) 2021 Chung Chan */

load(descriptive)$
load(draw)$


/****************
 helper functions 
 ****************/

/*
Simplify an expression.
*/
simplify(ex) := ev(fullratsimp(ex), simp)$

/*
Log base 2.
*/
log2(x):=log(x)/log(2)$

/*
Characteristic function.
*/
chi(p):=(if p then 1 else 0)$

/********************************
 Data construction/transformation
 ********************************/

/* Create data with feature names fns and a list lst of instances (list of 
   feature values). */
build_data_from_list(fns, lst):=apply('matrix, cons(fns,lst))$

/* Create data with feature names fns and a matrix M with rows as the 
   instances. */
build_data_from_matrix(fns, M):=build_data_from_list(fns, args(M))$

/* 
Generate data with feature names fns by generating the instances one-by-one by 
calling the function gen n times, each with an unique index as its argument 
from 1 to n. 
*/
build_data(fns, gen, n):=block(
    [lst:[]],
    for i:1 thru n do
    lst:endcons(
        block(
            [v:gen(i)],
            for j:1 thru length(fns) do
            v:subst(fns[j]=v[j], v),
            map(simplify, v)
        ),
        lst
    ),
    build_data_from_list(fns, lst)
)$

/*
Return the column index of the feature fn in the list fns of feature names.
The returned value is false if fn is not in fns.
*/
feature_index(fns, fn):=block(
    [i:1],
    for x in fns unless x=fn do i:i+1,
    if i<=length(fns) then i
)$

/*
Return the size (number of instances) of the data.
*/
size(data):=length(data)-1$

/*
Return the i-th instance of data as a list of feature values.
*/
get_data(data, i):=args(data)[i+1]$

/*
Return the feature names of the data.
*/
feature_names(data):=args(data)[1]$

/*
Return the matrix of all feature values.
*/
all_feature_values(data):=submatrix(1,data)$

/*
Return a list of feature values in data for the feature fn.
*/
feature_values(data, fn):=block(
    [i:feature_index(feature_names(data), fn)],
    rest(args(transpose(col(data,i)))[1],1)
)$

/*
Subsample data satisfying the condition cond, which is a boolean function that 
takes a unique index (1, 2, ...) as the argument.
*/
subsample_data(data, cond):=block(
    [
        fns:feature_names(data),
        fvs: args(all_feature_values(data)),
        lst:[]
    ],
    for i:1 thru size(data) do
    block(
        [
            r: fvs[i],
            c:cond(i)
        ],
        if simplify(psubst(map("=",fns,r),c)) then
        lst:endcons(r,lst)
    ),
    build_data_from_list(fns,lst)
)$

/*
Transform the features of data to nfns using the generator ngen.
*/
transform_features(data, nfns, ngen):=block(
    [
        lst:[], 
        fns:feature_names(data),
        fvs:args(all_feature_values(data))
    ],
    for i:1 thru size(data) do
    lst:endcons
    (
        block(
            [
                v:ngen(i),
                r:fvs[i]
            ],
            v:psubst(map("=", fns, r), v),
            for j:1 thru length(nfns) do
            v:subst(nfns[j]=v[j], v),
            map(simplify,v)
        ),
        lst
    ),
    build_data_from_list(nfns,lst)
)$

/*
Stack the list lst of data (vertically).
*/
stack_data([lst]):=build_data_from_list(
    feature_names(lst[1]),
    apply('append,map(lambda([D], args(all_feature_values(D))), lst))
)$

/*
Plot labeled data with x-axis xy[1] and y-axis xy[2]. 
*/
plot_labeled_data(data,xy,target):=
block(
    [
        Ds:split_data(data, target),
        D, lst:[]
    ],
    for i:1 thru length(Ds[2]) do
    (
        D:transform_features(Ds[2][i], xy, subst('xy=xy, lambda([i], xy))),
        lst:append(lst,[color=i, points(args(all_feature_values(D)))])
    ),
    draw2d(lst,xlabel=xy[1],ylabel=xy[2])
)$

/* Split data according to a splitting criterion X */
split_data(data, X):=block(
    [D, Djs:[]],
    D:transform_features(data, ['C], subst('X=X,lambda([i], [X]))),
    t:empirical(feature_values(D, 'C)),
    for j:1 thru length(t[1]) do
        block([Dj],
      